# 가스 공급량 수요예측 모델 개발 공모전

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

# 1.데이터 불러오기

In [2]:
df = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/한국가스공사_시간별 공급량_20181231.csv', encoding = 'utf-8')

In [3]:
weather13 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2013기상청.csv', encoding = 'utf-8')
weather14 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2014기상청.csv', encoding = 'utf-8')
weather15 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2015기상청.csv', encoding = 'utf-8')
weather16 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2016기상청.csv', encoding = 'utf-8')
weather17 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2017기상청.csv', encoding = 'utf-8')
weather18 = pd.read_csv('C:/Users/Administrator/데이터모음/가스공급량/2018기상청.csv', encoding = 'utf-8')

In [4]:
df

,연월일,시간,구분,공급량
0,2013-01-01,1,A,2497.129
1,2013-01-01,2,A,2363.265
2,2013-01-01,3,A,2258.505
3,2013-01-01,4,A,2243.969
4,2013-01-01,5,A,2344.105
...,...,...,...,...
368083,2018-12-31,20,H,681.033
368084,2018-12-31,21,H,669.961
368085,2018-12-31,22,H,657.941
368086,2018-12-31,23,H,610.953


In [5]:
weather13

,지점,지점명,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위),지면상태(지면상태코드),현상번호(국내식),지면온도(°C)
0,108,서울,2013-01-01 1:00,-8.5,NaN,4.0,57.0,1010.0,NaN,6.4,NaN,NaN,NaN,-3.4
1,108,서울,2013-01-01 2:00,-8.4,NaN,4.0,60.0,1009.4,NaN,6.4,NaN,NaN,NaN,-3.4
2,108,서울,2013-01-01 3:00,-8.1,NaN,3.4,58.0,1009.2,NaN,6.4,6.0,13.0,NaN,-3.4
3,108,서울,2013-01-01 4:00,-8.2,NaN,3.8,58.0,1008.2,NaN,6.4,6.0,NaN,NaN,-3.4
4,108,서울,2013-01-01 5:00,-8.2,NaN,4.4,61.0,1007.3,NaN,6.4,10.0,NaN,5.0,-3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,108,서울,2013-12-31 20:00,5.2,NaN,2.7,70.0,999.0,NaN,NaN,NaN,NaN,40.0,0.1
8756,108,서울,2013-12-31 21:00,4.9,NaN,3.9,64.0,999.6,NaN,NaN,6.0,1.0,40.0,0.1
8757,108,서울,2013-12-31 22:00,4.2,NaN,3.6,62.0,1000.4,NaN,NaN,NaN,NaN,40.0,0.1
8758,108,서울,2013-12-31 23:00,3.7,NaN,3.2,61.0,1001.2,NaN,NaN,NaN,NaN,40.0,0.1


# 2.데이터 확인하기

In [6]:
df.info()
print('-'*50)
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368088 entries, 0 to 368087
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   연월일     368088 non-null  object 
 1   시간      368088 non-null  int64  
 2   구분      368088 non-null  object 
 3   공급량     368088 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 11.2+ MB
--------------------------------------------------


연월일    0
시간     0
구분     0
공급량    0
dtype: int64

In [7]:
weather13.info()
print('-'*50)
weather13.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   지점            8760 non-null   int64  
 1   지점명           8760 non-null   object 
 2   일시            8760 non-null   object 
 3   기온(°C)        8760 non-null   float64
 4   강수량(mm)       930 non-null    float64
 5   풍속(m/s)       8760 non-null   float64
 6   습도(%)         8760 non-null   float64
 7   현지기압(hPa)     8760 non-null   float64
 8   일조(hr)        4795 non-null   float64
 9   적설(cm)        1079 non-null   float64
 10  전운량(10분위)     6807 non-null   float64
 11  지면상태(지면상태코드)  1463 non-null   float64
 12  현상번호(국내식)     3395 non-null   float64
 13  지면온도(°C)      8757 non-null   float64
dtypes: float64(11), int64(1), object(2)
memory usage: 958.2+ KB
--------------------------------------------------


지점                 0
지점명                0
일시                 0
기온(°C)             0
강수량(mm)         7830
풍속(m/s)            0
습도(%)              0
현지기압(hPa)          0
일조(hr)          3965
적설(cm)          7681
전운량(10분위)       1953
지면상태(지면상태코드)    7297
현상번호(국내식)       5365
지면온도(°C)           3
dtype: int64

In [8]:
weather13.info()
print('-'*50)
weather14.info()
print('-'*50)
weather15.info()
print('-'*50)
weather16.info()
print('-'*50)
weather17.info()
print('-'*50)
weather18.info()
print('-'*50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   지점            8760 non-null   int64  
 1   지점명           8760 non-null   object 
 2   일시            8760 non-null   object 
 3   기온(°C)        8760 non-null   float64
 4   강수량(mm)       930 non-null    float64
 5   풍속(m/s)       8760 non-null   float64
 6   습도(%)         8760 non-null   float64
 7   현지기압(hPa)     8760 non-null   float64
 8   일조(hr)        4795 non-null   float64
 9   적설(cm)        1079 non-null   float64
 10  전운량(10분위)     6807 non-null   float64
 11  지면상태(지면상태코드)  1463 non-null   float64
 12  현상번호(국내식)     3395 non-null   float64
 13  지면온도(°C)      8757 non-null   float64
dtypes: float64(11), int64(1), object(2)
memory usage: 958.2+ KB
--------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (t

# 3.데이터 전처리

In [9]:
#날씨 데이터 합치기

weather = pd.concat([weather13,weather14,weather15,weather16,weather17,weather18])
weather = weather.reset_index()

In [10]:
#필요없는 컬럼 삭제

weather = weather.drop(columns = ['index','지점','지점명'])

In [11]:
#결측치 0으로 대치
weather = weather.fillna(0)

In [12]:
#일시의 시간 표시 시간이라는 컬럼으로 빼기
from datetime import datetime


weather['일시'] = pd.to_datetime(weather['일시'])
weather['연월일'] = weather['일시'].dt.date
weather['시간'] = weather['일시'].dt.hour

In [13]:
#시간이 0 인거 24로 바꾸기
for i in tqdm(range(len(weather))):
    
    if weather['시간'][i] == 0:
        weather['시간'][i] = 24

  0%|                                                                                        | 0/52584 [00:00<?, ?it/s]<ipython-input-13-11f76a0debf5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['시간'][i] = 24
100%|█████████████████████████████████████████████████████████████████████████| 52584/52584 [00:01<00:00, 32631.86it/s]


In [14]:
weather['잡'] = 0

#시간이 24인거 하루 전으로 만들기
for i in tqdm(range(len(weather))):
    if weather['시간'][i] == 24:
        weather['잡'][i] = weather['일시'][i] +timedelta(days = -1)

#하루전으로 되돌린 날짜 연월일에 넣기
for x in tqdm(range(len(weather))):
    if weather['잡'][x] != 0:
        weather['연월일'][x] = str(weather['잡'][x])[:10]


weather = weather.drop(columns = '잡')

  0%|                                                                                        | 0/52584 [00:00<?, ?it/s]<ipython-input-14-b08108ef07ba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['잡'][i] = weather['일시'][i] +timedelta(days = -1)
  0%|                                                                                        | 0/52584 [00:00<?, ?it/s]<ipython-input-14-b08108ef07ba>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['연월일'][x] = str(weather['잡'][x])[:10]
100%|█████████████████████████████████████████████████████████████████████████| 52584/52584 [00:02<00:00, 25209.96it/s]


In [15]:
#df데이타와 weather 데이터 합치기

df['연월일'] = df['연월일'].astype('str')
weather['연월일'] = weather['연월일'].astype('str')


df = pd.merge(df, weather, on = ['연월일','시간'], how = 'left')

In [16]:
#구분 기준으로 결측치 확인
for i in df['구분'].unique():
    a = df[df['구분'] == i]
    print('-'*25,i,'-'*25)
    print(a.isnull().sum())
    print('-'*50)

------------------------- A -------------------------
연월일             0
시간              0
구분              0
공급량             0
일시              0
기온(°C)          0
강수량(mm)         0
풍속(m/s)         0
습도(%)           0
현지기압(hPa)       0
일조(hr)          0
적설(cm)          0
전운량(10분위)       0
지면상태(지면상태코드)    0
현상번호(국내식)       0
지면온도(°C)        0
dtype: int64
--------------------------------------------------
------------------------- B -------------------------
연월일             0
시간              0
구분              0
공급량             0
일시              0
기온(°C)          0
강수량(mm)         0
풍속(m/s)         0
습도(%)           0
현지기압(hPa)       0
일조(hr)          0
적설(cm)          0
전운량(10분위)       0
지면상태(지면상태코드)    0
현상번호(국내식)       0
지면온도(°C)        0
dtype: int64
--------------------------------------------------
------------------------- C -------------------------
연월일             0
시간              0
구분              0
공급량             0
일시              0
기온(°C)          0
강수량(mm)         0
풍속(m/s) 

In [17]:
df

,연월일,시간,구분,공급량,일시,기온(°C),강수량(mm),풍속(m/s),습도(%),현지기압(hPa),일조(hr),적설(cm),전운량(10분위),지면상태(지면상태코드),현상번호(국내식),지면온도(°C)
0,2013-01-01,1,A,2497.129,2013-01-01 01:00:00,-8.5,0.0,4.0,57.0,1010.0,0.0,6.4,0.0,0.0,0.0,-3.4
1,2013-01-01,2,A,2363.265,2013-01-01 02:00:00,-8.4,0.0,4.0,60.0,1009.4,0.0,6.4,0.0,0.0,0.0,-3.4
2,2013-01-01,3,A,2258.505,2013-01-01 03:00:00,-8.1,0.0,3.4,58.0,1009.2,0.0,6.4,6.0,13.0,0.0,-3.4
3,2013-01-01,4,A,2243.969,2013-01-01 04:00:00,-8.2,0.0,3.8,58.0,1008.2,0.0,6.4,6.0,0.0,0.0,-3.4
4,2013-01-01,5,A,2344.105,2013-01-01 05:00:00,-8.2,0.0,4.4,61.0,1007.3,0.0,6.4,10.0,0.0,5.0,-3.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,H,681.033,2018-12-31 20:00:00,-3.7,0.0,0.8,39.0,1024.9,0.0,0.0,5.0,0.0,0.0,-3.0
368084,2018-12-31,21,H,669.961,2018-12-31 21:00:00,-4.6,0.0,1.1,44.0,1024.8,0.0,0.0,0.0,0.0,0.0,-4.1
368085,2018-12-31,22,H,657.941,2018-12-31 22:00:00,-5.4,0.0,1.3,46.0,1024.4,0.0,0.0,0.0,0.0,0.0,-5.0
368086,2018-12-31,23,H,610.953,2018-12-31 23:00:00,-5.2,0.0,1.6,47.0,1024.6,0.0,0.0,0.0,0.0,0.0,-5.1
